In [ ]:
# This notebook exports a Catchment_attributes.csv file for Basin Delineation A

In [ ]:
import pandas as pds
import numpy as np
from pathlib import Path
import geopandas as gpd
import geopandas as gpds

In [4]:
# Define paths to files
gauges_path = Path(r'C:\Users\hordurbhe\Dropbox\UW\lamah_ice\GIS\watersheds\final_watersheds\gauges_with_splitted_included.shp') 
wsheds_path = Path(r'C:\Users\hordurbhe\Dropbox\UW\lamah_ice\GIS\watersheds\final_watersheds\final_watersheds\Basins_A.shp')
valid_years_path = Path(r"C:\Users\hordurbhe\Dropbox\UW\lamah_ice\lamah_ice\A_basins_total_upstrm\1_attributes\final_attributes\valid_years_10_perc_tol_1981_2018_ravII.csv")

streamflow_ind_path = Path(r"C:\Users\hordurbhe\Dropbox\UW\lamah_ice\lamah_ice\A_basins_total_upstrm\1_attributes\final_attributes\hydro_sign_10_perc_tol_only_more_than_2_years_1981_2018_ravII.csv")
streamflow_ind_path_unfilt = Path(r"C:\Users\hordurbhe\Dropbox\UW\lamah_ice\lamah_ice\A_basins_total_upstrm\1_attributes\final_attributes\hydro_sign_10_perc_tol_only_more_than_2_years_1981_2018_ravII_unfiltered.csv")
soil_path = Path(r"C:\Users\hordurbhe\Dropbox\UW\lamah_ice\lamah_ice\A_basins_total_upstrm\1_attributes\final_attributes\soil_attrs_basins_A.csv")
human_infl_path = Path(r"C:\Users\hordurbhe\Dropbox\UW\lamah_ice\lamah_ice\A_basins_total_upstrm\1_attributes\final_attributes\human_influence_cleaned_reordered.csv")
NI_geo_attrs_path = Path(r"C:\Users\hordurbhe\Dropbox\UW\lamah_ice\lamah_ice\A_basins_total_upstrm\1_attributes\final_attributes\NI_geo_attrs_basins_A.csv")
topo_attrs_path = Path(r"C:\Users\hordurbhe\Dropbox\UW\lamah_ice\lamah_ice\A_basins_total_upstrm\1_attributes\final_attributes\topo_attrs_Basins_A.csv")
clim_ind_path = Path(r"C:\Users\hordurbhe\Dropbox\UW\lamah_ice\lamah_ice\A_basins_total_upstrm\1_attributes\final_attributes\Clim_ind_1989_2009_ravII.csv")
clim_ind_path_ERA5L = Path(r"C:\Users\hordurbhe\Dropbox\UW\lamah_ice\lamah_ice\A_basins_total_upstrm\1_attributes\final_attributes\Clim_ind_1989_2009_ERA5L.csv")
water_balance_path = Path(r"C:\Users\hordurbhe\Dropbox\UW\lamah_ice\lamah_ice\A_basins_total_upstrm\1_attributes\final_attributes\water_balance_with_era5L_and_ravII.csv")
water_balance_path_including_unchecked = Path(r"C:\Users\hordurbhe\Dropbox\UW\lamah_ice\lamah_ice\A_basins_total_upstrm\1_attributes\final_attributes\water_balance_with_era5L_and_ravII_unfiltered.csv")
# Meteorological variable means, for 1981-2018
met_means_path_1981 = Path(r"C:\Users\hordurbhe\Documents\Vinna\lamah\lamah_ice\lamah_ice\A_basins_total_upstrm\1_attributes\meteorological_data_means_1981_to_2018.csv")
# Meteorological variable means, for 1991-2018
met_means_path_1991 = Path(r"C:\Users\hordurbhe\Documents\Vinna\lamah\lamah_ice\lamah_ice\A_basins_total_upstrm\1_attributes\meteorological_data_means_1991_to_2018.csv")

valid_streamfl_data_1981_2021_unfilt_path = Path(r"C:\Users\hordurbhe\Dropbox\UW\lamah_ice\lamah_ice\A_basins_total_upstrm\1_attributes\final_attributes\valid_years_10_perc_tol_1981_2021_final_unfiltered_final_final.csv")
valid_streamfl_data_1981_2021_filt_path = Path(r"C:\Users\hordurbhe\Dropbox\UW\lamah_ice\lamah_ice\A_basins_total_upstrm\1_attributes\final_attributes\valid_years_10_perc_tol_1981_2021_final_filt_final.csv")
# Þessar skrár eru búnar til hér:
# http://localhost:8888/notebooks/Dropbox/UW/lamah_ice/Notebooks/make_streamflow_data_availability_plot.ipynb

In [5]:
# Read gauges shapefile
gauges = gpds.read_file(gauges_path)
gauges = gauges.set_index('id')
cols = ['st_numer', 'st_nafn', 'vhm_numer', 'vatnsfall', 'gagnaeigan', 'stada', 'upphaf_rek','dags_aflag', 'elevation']
gauges_reduced = gauges[cols]
gauges_reduced.columns = ['V_no','name','VHM_no','river','data_owner','status','upphaf_rek','dags_aflag', 'elevation']

# Read watershed attributes (calculated in QGIS)
wsheds_attrs = gpds.read_file(wsheds_path)
wsheds_attrs.index = wsheds_attrs['id']
wsheds_attrs = wsheds_attrs.drop(columns=['id'])
wsheds_attrs['lat'] = gauges.geometry.y
wsheds_attrs['lon'] = gauges.geometry.x
# Drop columns from wsheds_attrs that have no values
wsheds_attrs.dropna(axis=1, how='all', inplace=True)
# Extract only the relevant columns from wsheds_attrs
wsheds_attrs = wsheds_attrs[['gc_dom', 'strm_dens', 'mvert_dist', 'gc_ss_fra', 
                             'area_calc', 'gc_wb_fra', 'litho_vapy', 'gc_py_fra', 
                             'litho_dom', 'gc_ig_fra', 'elon_ratio',  'lon','lat',
                             'litho_pb', 'gc_sm_fra', 'lc_dom', 'mvert_ang', 
                             'gc_vb_fra', 'gc_mt_fra', 'litho_vbsr', 'gc_pi_fra', 
                              'gc_va_fra', 'gc_su_fra', 'litho_pavr', 'litho_vbpy', 
                             'gc_pb_fra', 'litho_vb',  'gc_sc_fra', 'gc_pa_fra','lai_max', 
                             'lai_diff', 'ndvi_max', 'ndvi_min','gvf_max', 'gvf_diff','geometry']] 
# Read streamflow indices
valid_years_df = pds.read_csv(valid_years_path)
valid_years_df = valid_years_df.set_index('Unnamed: 0')
valid_years_df.index = valid_years_df.index.rename('id')


# Read streamflow indices
streamflow_indices = pds.read_csv(streamflow_ind_path)
streamflow_indices = streamflow_indices.set_index('Unnamed: 0')
streamflow_indices.index = streamflow_indices.index.rename('id')

streamflow_indices_unfilt = pds.read_csv(streamflow_ind_path_unfilt)
streamflow_indices_unfilt = streamflow_indices_unfilt.set_index('Unnamed: 0')
streamflow_indices_unfilt.index = streamflow_indices_unfilt.index.rename('id')
newcols = [x+'_unfilt' for x in streamflow_indices_unfilt.columns]
streamflow_indices_unfilt.columns = newcols

# Read the soil attributes where depth to bedrock is calculated by excluding the glaciated regions
soil_indices = pds.read_csv(soil_path)
soil_indices = soil_indices.set_index('Unnamed: 0')
soil_indices.index = soil_indices.index.rename('id')

NI_geo_attrs = pds.read_csv(NI_geo_attrs_path)
NI_geo_attrs = NI_geo_attrs.set_index('ID')
NI_geo_attrs.index = NI_geo_attrs.index.rename('id')

# Read the (human) influence attributes
influence = pds.read_csv(human_infl_path,sep='\t',encoding='UTF-16')
influence = influence.set_index('id')

# Read csv with water balance stats (created here: http://localhost:8888/notebooks/Dropbox/UW/lamah_ice/Notebooks/streamflow_indices_in_python_and_water_balance_stats_valid_years_only.ipynb)
water_balance_df = pds.read_csv(water_balance_path)
water_balance_df = water_balance_df.set_index('Unnamed: 0')

water_balance_df_including_unchecked = pds.read_csv(water_balance_path_including_unchecked)
water_balance_df_including_unchecked = water_balance_df_including_unchecked.set_index('Unnamed: 0')
newcols = [x+'_unfilt' for x in water_balance_df_including_unchecked.columns]
water_balance_df_including_unchecked.columns = newcols

# Topographical attributes
topo_attrs = pds.read_csv(topo_attrs_path,sep=';')
topo_attrs = topo_attrs.set_index('id')

# Read ERA-Land data
era5_data = pds.read_csv(clim_ind_path,sep=';') 
era5_data.index=era5_data['ID']
era5_data.index = era5_data.index.rename('id')

# Read met data means
met_means_1981 = pds.read_csv(met_means_path_1981,sep=',')
met_means_1981.set_index('Unnamed: 0', inplace=True)
met_means_1981.index = met_means_1981.index.rename('id')

# Read met data means
met_means_1991 = pds.read_csv(met_means_path_1991,sep=',')
met_means_1991.set_index('Unnamed: 0', inplace=True)
met_means_1991.index = met_means_1981.index.rename('id')

# Read valid year counts for unfiltered data
valid_streamfl_data_1981_2021_unfilt = pds.read_csv(valid_streamfl_data_1981_2021_unfilt_path,sep=',')
valid_streamfl_data_1981_2021_unfilt.set_index('Unnamed: 0', inplace=True)
valid_streamfl_data_1981_2021_unfilt.index = valid_streamfl_data_1981_2021_unfilt.index.rename('id')

# Read valid year counts for filtered data
valid_streamfl_data_1981_2021_filt = pds.read_csv(valid_streamfl_data_1981_2021_filt_path,sep=',')
valid_streamfl_data_1981_2021_filt.set_index('Unnamed: 0', inplace=True)
valid_streamfl_data_1981_2021_filt.index = valid_streamfl_data_1981_2021_filt.index.rename('id')

# ERA5-Land climate indices
clim_ind_era5L = pds.read_csv(clim_ind_path_ERA5L,sep=';') 
clim_ind_era5L.set_index('ID',inplace=True)
clim_ind_era5L.index = clim_ind_era5L.index.rename('id')
newcols = [x+'_ERA5L' for x in clim_ind_era5L.columns]
clim_ind_era5L.columns = newcols

# Read corine files
corine_path = Path(r'C:\Users\hordurbhe\Documents\Vinna\lamah\lamah_ice\corine\corine_watersheds_fractions_python\Basins_A\corine_2018.csv')
corine_attrs = pds.read_csv(corine_path)
corine_attrs.set_index('Watershed ID',inplace=True)
corine_attrs = corine_attrs[['agr_fra','bare_fra','forest_fra','glac_fra','lake_fra','urban_fra','scrub_fra','wetl_fra']]

# Read glaciers and calculate g_lat and g_lon
# We read the shapefile containing the glacier area or areas in each catchment
glaciers_path = Path(r'C:\Users\hordurbhe\Documents\Vinna\lamah\lamah_ice\lamah_ice\A_basins_total_upstrm\3_shapefiles\glaciers_in_catchments.shp')
glaciers_ = gpds.read_file(glaciers_path)
glaciers_ = glaciers_.set_index('id')

# First we calculate the lat and lon of the glacier centroid
glaciers_['centroid'] = glaciers_.centroid
glaciers_['g_lon'] = glaciers_['centroid'].x
glaciers_['g_lat'] = glaciers_['centroid'].y

# Extract glacier attributes from Hannesdóttir et al.

In [6]:
import pandas as pd
import glob
from pathlib import Path

# Define the path to the directory containing the files
path = Path(r'C:\Users\hordurbhe\Documents\Vinna\lamah\lamah_ice\lamah_ice\A_basins_total_upstrm\2_timeseries\annual\glacier_timeseries')
# Search for all CSV files in the directory
fnames = glob.glob(str(path / '*.csv'))

# Initialize an empty dictionary to store data
glac_percent_2019_dict = {}

# Loop through each file in the directory
for filepath in fnames:
    # Extract the catchment ID from the file name
    ID = Path(filepath).stem.split('_')[1]  # Assumes the ID is before the first underscore in the file name
    
    # Read the CSV file into a DataFrame
    df = pd.read_csv(filepath, index_col=0,sep=';')  # Assuming the first column is the year or another index
    # Check if 2019 data exists
    if 2019 in df.index:
        # Extract glac_area_perc and glac_area_km2 values for the year 2019
        glac_area_perc = df.loc[2019, 'g_frac_dyn'] / 100
        glac_area_km2 = df.loc[2019, 'g_area_dyn']
        # Update the dictionary with the extracted values
        glac_percent_2019_dict[ID] = {'g_frac': glac_area_perc, 'g_area': glac_area_km2}
    else:
        print('no')

# Convert the dictionary to a DataFrame
df_summary = pd.DataFrame.from_dict(glac_percent_2019_dict, orient='index')
# Set the DataFrame's index name to 'catchment ID' for clarity
df_summary.index.name = 'catchment ID'
df_summary.index = df_summary.index.astype(int)
df_summary = df_summary.sort_index().fillna(0)
df_glacier_hannesd = df_summary

# Merge all dataframes

In [7]:
# Base DataFrame
merged_df = wsheds_attrs.sort_index()

# List of DataFrames to merge using an outer join initially
dfs_to_merge_outer = [
    soil_indices, NI_geo_attrs, influence, topo_attrs, met_means_1991, met_means_1981, corine_attrs, 
    df_glacier_hannesd, valid_streamfl_data_1981_2021_filt, valid_streamfl_data_1981_2021_unfilt, clim_ind_era5L
]

# Merge with outer join
for df in dfs_to_merge_outer:
    merged_df = merged_df.merge(df.sort_index(), left_index=True, right_index=True, how='outer')

# List of DataFrames to merge using left join
dfs_to_merge_left = [
    streamflow_indices, streamflow_indices_unfilt, era5_data, water_balance_df_including_unchecked, 
    water_balance_df, gauges_reduced, glaciers_[['g_lon', 'g_lat']]
]

# Merge with left join
for df in dfs_to_merge_left:
    merged_df = merged_df.merge(df.sort_index(), left_index=True, right_index=True, how='left')

# Update geometry from gauges if necessary
merged_df['geometry'] = gauges.geometry

# Check the final type (should be a DataFrame)
type(merged_df)

geopandas.geodataframe.GeoDataFrame

In [11]:
merged_dff = merged_df.copy()

In [38]:
attrs_to_export  =  ['area_calc',
                     'elev_mean',
                     'elev_med',
                     'elev_ran',
                     'elev_std',
                     'slope_mean', 'mvert_dist','mvert_ang',
                     'asp_mean','elon_ratio','strm_dens',
                     
                     'p_mean',
                     'pet_mean',
                     'aridity',
                     'p_seasonality',
                     'frac_snow',
                     'high_prec_freq',
                     'high_prec_dur',
                     'high_prec_timing',
                     'low_prec_freq',
                     'low_prec_dur',
                     'low_prec_timing',
                     
                     'p_mean_ERA5L',
                     'pet_mean_ERA5L',
                     'aridity_ERA5L',
                     'p_seasonality_ERA5L',
                     'frac_snow_ERA5L',
                     'high_prec_freq_ERA5L',
                     'high_prec_dur_ERA5L',
                     'high_prec_timing_ERA5L',
                     'low_prec_freq_ERA5L',
                     'low_prec_dur_ERA5L',
                     'low_prec_timing_ERA5L',
                     
                     'q_mean', 'runoff_ratio', 'hfd_mean', 'slope_fdc', 'baseflow_index_ladson', 'stream_elas', 'high_q_freq', 'high_q_dur', 'Q95', 'low_q_freq', 'low_q_dur', 'Q5',
                     
                     'lc_dom','agr_fra','bare_fra', 'forest_fra', 'glac_fra',
                     'lake_fra','urban_fra','scrub_fra','wetl_fra',
                     
                     'lai_max',
                     'lai_diff',
                     'ndvi_max',
                     'ndvi_min',
                     'gvf_max',
                     'gvf_diff',
                     
                     'bedrk_dep','root_dep','soil_poros','soil_tawc',
                     'sand_fra','silt_fra','clay_fra','grav_fra','oc_fra',
                     
                     'gc_dom','gc_pa_fra','gc_pb_fra', 'gc_va_fra','gc_vb_fra',
                      'litho_dom','litho_pavr','litho_pb','litho_vapy','litho_vbsr','litho_vbpy', 'litho_vb',
                     
                     'g_dom_NI',
                     'g621_fra',
                     'g701_fra',
                     'g743_fra',
                     'g746_fra',
                     'gbinn_fra',
                     'ggnew_fra',
                     'ggold_fra',
                     'ghraun_fra',
                     'gbnew_fra',
                     'gbold_fra',
                     'gmob_fra',
                     'gsgos_fra',
                     'gsinn_fra',
                     'gsn_fra',
                     'gsnew_fra',
                     'gsold_fra',
                     
                     'g_frac',
                     'g_area',
                     'g_mean_el',
                     'g_min_el',
                     'g_max_el',
                     'g_aspect',
                     'g_slope',
                     'g_slopel20',
                     'g_lon',
                     'g_lat',
                    
                     'degimpact',
                     'typimpact']

hydro_indices = ['q_mean',
                     'runoff_ratio','runoff_ratio_ERA5L',
                     'stream_elas',
                     'slope_fdc',
                     'baseflow_index_ladson',
                     'hfd_mean',
                     'Q5',
                     'Q95',
                     'high_q_freq',
                     'high_q_dur',
                     'low_q_freq',
                     'low_q_dur',
                     'zero_q_freq','year_count']

hydro_indices_unfilt = ['q_mean_unfilt',
                         'runoff_ratio_unfilt',
                         'stream_elas_unfilt',
                         'slope_fdc_unfilt',
                         'baseflow_index_ladson_unfilt',
                         'hfd_mean_unfilt',
                         'Q5_unfilt',
                         'Q95_unfilt',
                         'high_q_freq_unfilt',
                         'high_q_dur_unfilt',
                         'low_q_freq_unfilt',
                         'low_q_dur_unfilt',
                         'zero_q_freq_unfilt','year_count_unfilt']


# Define the dictionary for column renaming
rename_dict = {
    'p_seasonality': 'p_season',
    'p_seasonality_ERA5L': 'p_season_ERA5L',
    'high_prec_freq': 'high_prec_fr',
    'high_prec_freq_ERA5L': 'high_prec_fr_ERA5L',
    'low_prec_freq': 'lo_prec_fr',
    'low_prec_freq_ERA5L': 'lo_prec_fr_ERA5L',
    'high_prec_dur': 'high_prec_du',
    'low_prec_dur': 'low_prec_du',
    'high_prec_dur_ERA5L': 'high_prec_du_ERA5L',
    'low_prec_dur_ERA5L': 'low_prec_du_ERA5L',
    'litho_vapy': 'gc_23_vapy',
    'litho_dom': 'gc_23_dom',
    'litho_pavr': 'gc_23_pavr',
    'litho_pb': 'gc_23_pb',
    'litho_vbsr': 'gc_23_vbsr',
    'litho_vbpy': 'gc_23_vbpy',
    'litho_vb': 'gc_23_vb',
    'pet_mean': 'ref_et_mean',
    'Degree': 'degimpact'
}

# Update your attrs_to_export list to reflect these new names
attrs_to_export_updated = [rename_dict.get(attr, attr) for attr in attrs_to_export]

# Now, rename the columns in the DataFrame
merged_dff_renamed = merged_dff.rename(columns=rename_dict)
merged_dff_renamed.index = merged_dff_renamed.index.rename('id')
# Finally, export the DataFrame with updated column names to CSV
attributes_path = Path(r"C:\Users\hordurbhe\Documents\Vinna\lamah\lamah_ice\lamah_ice\A_basins_total_upstrm\1_attributes\Catchment_attributes.csv")
merged_dff_renamed[attrs_to_export_updated].round(3).to_csv(attributes_path,sep=';')

attributes_path = Path(r"C:\Users\hordurbhe\Documents\Vinna\lamah\lamah_ice\lamah_ice\A_basins_total_upstrm\1_attributes\Catchment_attributes.csv")
merged_dff[attrs_to_export].round(3).to_csv(attributes_path)

hydro_indices_filtered_path = Path(r"C:\Users\hordurbhe\Documents\Vinna\lamah\lamah_ice\lamah_ice\D_gauges\1_attributes\hydro_indices_filtered_obs.csv")
merged_dff[hydro_indices].round(3).to_csv(hydro_indices_filtered_path)

hydro_indices_unfiltered_path = Path(r"C:\Users\hordurbhe\Documents\Vinna\lamah\lamah_ice\lamah_ice\D_gauges\1_attributes\hydro_indices_unfiltered_obs.csv")
unfilt = merged_dff[hydro_indices_unfilt]
unfilt.columns = hydro_indices
unfilt.round(3).to_csv(hydro_indices_unfiltered_path)

In [33]:
merged_dff_renamed.index = merged_dff_renamed.index.rename('id')
len(merged_dff_renamed['runoff_ratio_unfilt'].dropna())

97

In [29]:
len(merged_dff_renamed['runoff_ratio'].dropna())

73

In [30]:
# merged_dff_renamed is the GeoDataFrame and attrs_to_export_gpds_version is the list of columns to export
# First, ensure attrs_to_export_gpds_version has unique values
attrs_to_export_gpds_version = attrs_to_export_updated.copy()
attrs_to_export_gpds_version = attrs_to_export_gpds_version+['P_ravII_1981','P_ravII_1991','P_ERA5L_1981','P_ERA5L_1991','P_carra_1991','ET_ravII_1981','ET_ERA5L_1981','PET_ravII_1981','PET_ERA5L_1981','year_count_1981_2021','year_count_1981_2021_unfilt','Q','P_ERA5L','PET_ERA5L','ET_ERA5L', 'P_rav','PET_rav','ET_rav']+['q_mean', 'runoff_ratio', 'hfd_mean', 'slope_fdc', 'baseflow_index_ladson', 'stream_elas', 'high_q_freq', 'high_q_dur', 'Q95', 'low_q_freq', 'low_q_dur', 'Q5'] + ['q_mean_unfilt', 'runoff_ratio_unfilt', 'hfd_mean_unfilt', 'slope_fdc_unfilt', 'baseflow_index_ladson_unfilt', 'stream_elas_unfilt', 'high_q_freq_unfilt', 'high_q_dur_unfilt', 'Q95_unfilt', 'low_q_freq_unfilt', 'low_q_dur_unfilt', 'Q5_unfilt']

attrs_to_export_gpds_version_unique = list(set(attrs_to_export_gpds_version))

# Ensure 'geometry' column is in the list of attributes to export
if 'geometry' not in attrs_to_export_gpds_version:
    attrs_to_export_gpds_version_unique.append('geometry')

# Create a new GeoDataFrame with only the desired columns
gdf_to_export = merged_dff_renamed[attrs_to_export_gpds_version_unique]

# Make sure gdf_to_export is recognized as a GeoDataFrame, especially if not already
if not isinstance(gdf_to_export, gpd.GeoDataFrame):
    gdf_to_export = gpd.GeoDataFrame(gdf_to_export, geometry='geometry')

# Export to GeoPackage
gdf_to_export.to_file("C:/Users/hordurbhe/Documents/Vinna/lamah/lamah_ice/lamah_ice/F_appendix/1_codes/plots/Catchment_and_gauge_attributes_used_for_plotting.gpkg", driver='GPKG')